In [2]:
import pandas as pd
import numpy as np
import re
import json

def parse_model_and_variant(model_name):
    """
    Tách model_name thành model và variant
    """
    if not model_name or pd.isna(model_name):
        return "", ""
    
    model_name_str = str(model_name)
    
    # Trường hợp FI + số cc
    if " FI " in model_name_str:
        parts = model_name_str.split(" FI ")
        return parts[0], "FI " + parts[1]
    
    # Kiểm tra nếu kết thúc bằng số cc
    cc_match = re.search(r'\s+(\d+)(?:\s*cc)?$', model_name_str)
    if cc_match:
        return model_name_str[:cc_match.start()], cc_match.group().strip()
    
    # Kiểm tra nếu kết thúc bằng "ABS"
    abs_match = re.search(r'\s+(ABS|R ABS)$', model_name_str)
    if abs_match:
        return model_name_str[:abs_match.start()], abs_match.group().strip()
    
    # Kiểm tra nếu có "FI" ở cuối
    fi_match = re.search(r'\s+FI$', model_name_str)
    if fi_match:
        return model_name_str[:fi_match.start()], "FI"
    
    # Nếu không tìm thấy pattern nào phù hợp
    return model_name_str, ""

def add_model_variant_columns(df, model_name_col, current_model_col=None, current_variant_col=None):
    """
    Thêm cột model_updated và variant_updated vào DataFrame
    """
    # Sao chép DataFrame
    result_df = df.copy()
    
    # Nếu đã có cột model và variant, sử dụng lại
    if current_model_col and current_variant_col and current_model_col in df.columns and current_variant_col in df.columns:
        result_df['model_updated'] = df[current_model_col]
        result_df['variant_updated'] = df[current_variant_col]
        return result_df
    
    # Khởi tạo cột mới
    result_df['model_updated'] = ""
    result_df['variant_updated'] = ""
    
    # Xử lý các dòng có model_name
    if model_name_col in df.columns:
        mask = ~df[model_name_col].isna()
        model_variant_pairs = df.loc[mask, model_name_col].apply(parse_model_and_variant)
        
        result_df.loc[mask, 'model_updated'] = model_variant_pairs.apply(lambda x: x[0])
        result_df.loc[mask, 'variant_updated'] = model_variant_pairs.apply(lambda x: x[1])
    
    return result_df

def process_bike_catalog(file_path):
    """
    Xử lý file bike_catalog.csv
    """
    # Đọc file CSV
    try:
        df = pd.read_csv(file_path)
        print(f"Đọc file {file_path} thành công: {len(df)} dòng")
    except Exception as e:
        print(f"Lỗi khi đọc file {file_path}: {e}")
        return pd.DataFrame()
    
    # Thêm cột model_updated và variant_updated
    df = add_model_variant_columns(df, 'model_name')
    
    # Phân tích cột available_years
    def extract_latest_year(years_str):
        if not years_str or pd.isna(years_str):
            return np.nan
        
        try:
            # Cố gắng parse chuỗi như một list Python
            years_list = eval(years_str)
            return max(int(year) for year in years_list)
        except:
            # Nếu không thể parse, tìm tất cả các số có 4 chữ số
            years_match = re.findall(r'\d{4}', str(years_str))
            if years_match:
                return max(int(year) for year in years_match)
        
        return np.nan
    
    df['latest_year'] = df['available_years'].apply(extract_latest_year)
    
    return df

def process_price_data(file_path):
    """
    Xử lý file vnexpress_price.csv
    """
    # Đọc file CSV
    try:
        df = pd.read_csv(file_path)
        print(f"Đọc file {file_path} thành công: {len(df)} dòng")
    except Exception as e:
        print(f"Lỗi khi đọc file {file_path}: {e}")
        return pd.DataFrame()
    
    # Thêm cột model_updated và variant_updated
    df = add_model_variant_columns(df, 'model_name')
    
    return df

def extract_info_from_catalog_price(bike_catalog_df, price_df):
    """
    Trích xuất thông tin từ bike_catalog_df và price_df để mapping với used_bike_df
    """
    # Tạo DataFrame kết quả
    info_df = pd.DataFrame()
    
    # Danh sách tất cả các model-variant để mapping
    model_info = {}
    
    # Thêm models từ bike_catalog
    if not bike_catalog_df.empty:
        for _, row in bike_catalog_df.iterrows():
            brand = row.get('brand_name', '')
            model = row.get('model_updated', '')
            variant = row.get('variant_updated', '')
            category = row.get('model_category', '')
            years = row.get('available_years', '')
            
            if brand and model:
                key = (brand, model, variant)
                if key not in model_info:
                    model_info[key] = {
                        'brand': brand,
                        'model': model,
                        'variant': variant,
                        'category': category,
                        'years': years,
                        'catalog_source': True,
                        'price_source': False,
                        'new_price': None
                    }
    
    # Thêm models từ price_data
    if not price_df.empty:
        for _, row in price_df.iterrows():
            brand = row.get('brand_name', '')
            model = row.get('model_updated', '')
            variant = row.get('variant_updated', '')
            price_min = row.get('price_min')
            price_max = row.get('price_max')
            
            if brand and model:
                key = (brand, model, variant)
                if key in model_info:
                    model_info[key]['price_source'] = True
                    model_info[key]['new_price'] = price_min
                else:
                    model_info[key] = {
                        'brand': brand,
                        'model': model,
                        'variant': variant,
                        'category': None,
                        'years': None,
                        'catalog_source': False,
                        'price_source': True,
                        'new_price': price_min
                    }
    
    # Tính giá trung bình cho mỗi model
    model_avg_prices = {}
    
    # Tính giá trung bình của new_price cho mỗi model
    for key, data in model_info.items():
        brand, model, _ = key
        model_key = (brand, model)
        
        if data['new_price'] is not None:
            if model_key not in model_avg_prices:
                model_avg_prices[model_key] = {'total': 0, 'count': 0}
            
            model_avg_prices[model_key]['total'] += data['new_price']
            model_avg_prices[model_key]['count'] += 1
    
    # Tính giá trung bình
    for model_key, data in model_avg_prices.items():
        if data['count'] > 0:
            data['avg_price'] = data['total'] / data['count']
        else:
            data['avg_price'] = None
    
    # Thêm thông tin giá model trung bình
    for key, data in model_info.items():
        brand, model, variant = key
        model_key = (brand, model)
        
        if model_key in model_avg_prices:
            data['new_price_model_avg'] = model_avg_prices[model_key]['avg_price']
        else:
            data['new_price_model_avg'] = None
    
    # Chuyển dictionary thành DataFrame
    info_data = []
    for key, data in model_info.items():
        info_data.append(data)
    
    info_df = pd.DataFrame(info_data)
    
    return info_df, model_avg_prices

# Main execution
if __name__ == "__main__":
    # Đường dẫn đến các file
    bike_catalog_path = "../bike_catalog.csv"
    price_data_path = "../vnexpress_price.csv"
    output_path = "catalog_price_info.csv"
    
    # Xử lý catalog
    bike_catalog_df = process_bike_catalog(bike_catalog_path)
    
    # Xử lý dữ liệu giá
    price_df = process_price_data(price_data_path)
    
    # Trích xuất thông tin
    info_df, model_avg_prices = extract_info_from_catalog_price(bike_catalog_df, price_df)
    
    # Lưu kết quả ra file CSV
    info_df.to_csv(output_path, index=False)
    print(f"Đã lưu thông tin model và giá xe mới vào file {output_path}")
    
    # Lưu model_avg_prices thành CSV
    avg_prices_data = []
    for (brand, model), data in model_avg_prices.items():
        avg_prices_data.append({
            'brand': brand,
            'model': model,
            'average_price': data.get('avg_price')
        })
    
    avg_prices_df = pd.DataFrame(avg_prices_data)
    avg_prices_df.to_csv(output_path.replace('.csv', '_avg_prices.csv'), index=False)
    print(f"Đã lưu giá trung bình theo model vào file {output_path.replace('.csv', '_avg_prices.csv')}")

Đọc file ../bike_catalog.csv thành công: 402 dòng
Đọc file ../vnexpress_price.csv thành công: 199 dòng
Đã lưu thông tin model và giá xe mới vào file catalog_price_info.csv
Đã lưu giá trung bình theo model vào file catalog_price_info_avg_prices.csv


In [1]:
import pandas as pd
import numpy as np
import re
from datetime import datetime

def extract_price_value(price_str):
    """
    Chuyển đổi chuỗi giá thành giá trị số (đơn vị: triệu đồng)
    """
    if not price_str or pd.isna(price_str):
        return np.nan
    
    try:
        # Loại bỏ ký tự không phải số và dấu chấm/phẩy
        price_numeric = re.sub(r'[^\d.,]', '', str(price_str))
        # Thay thế dấu chấm bằng rỗng và dấu phẩy bằng dấu chấm
        price_numeric = price_numeric.replace('.', '').replace(',', '.')
        
        price_value = float(price_numeric)
        
        # Chuyển đổi thành đơn vị triệu đồng nếu giá trị lớn
        if price_value > 1000:  # Nếu giá trị lớn hơn 1000, giả sử đơn vị là nghìn đồng
            price_value = price_value / 1000000  # Chuyển từ đồng sang triệu đồng
        
        return round(price_value, 2)
    except:
        return np.nan

def process_used_bike_data(file_path):
    """
    Xử lý file input_xe_cu.csv
    """
    # Đọc file CSV
    try:
        df = pd.read_csv(file_path)
        print(f"Đọc file {file_path} thành công: {len(df)} dòng")
    except Exception as e:
        print(f"Lỗi khi đọc file {file_path}: {e}")
        return pd.DataFrame()
    
    # Chuyển đổi giá thành giá trị số
    df['price_value'] = df['price'].apply(extract_price_value)
    
    # Xử lý cột model
    # Sửa trường hợp "Dòng khác" từ model bằng cách trích xuất từ title
    def extract_model_from_title(title, current_model):
        if current_model != "Dòng khác" and not pd.isna(current_model):
            return current_model
        
        if pd.isna(title):
            return current_model
        
        title_lower = title.lower()
        model_patterns = {
            'wave': 'Wave',
            'dream': 'Dream',
            'future': 'Future',
            'blade': 'Blade',
            'sh': 'SH',
            'vision': 'Vision',
            'lead': 'Lead',
            'air blade': 'Air Blade',
            'pcx': 'PCX',
            'winner': 'Winner',
            'exciter': 'Exciter',
            'sirius': 'Sirius',
            'jupiter': 'Jupiter',
            'nozza': 'Nozza',
            'grande': 'Grande',
            'janus': 'Janus',
            'vespa': 'Vespa',
            'liberty': 'Liberty',
            'primavera': 'Primavera',
            'sprint': 'Sprint',
            'gsx': 'GSX',
            'raider': 'Raider',
            'satria': 'Satria',
            'burgman': 'Burgman'
        }
        
        for pattern, model in model_patterns.items():
            if pattern in title_lower:
                return model
        
        return current_model
    
    df['model_updated'] = df.apply(lambda row: extract_model_from_title(row['title'], row['model']), axis=1)
    
    # Extract variant từ title cho các xe còn thiếu variant
    def extract_variant_from_title(title):
        if pd.isna(title):
            return ""
        
        title_lower = title.lower()
        
        # Trích xuất phiên bản
        variant_keywords = ['abs', 'cbs', 'fi', 'idling stop', 'smart key']
        variant_parts = []
        
        for keyword in variant_keywords:
            if keyword in title_lower:
                variant_parts.append(keyword.upper())
        
        # Trích xuất cc (ví dụ: 150i, 125cc)
        cc_match = re.search(r'\b(\d+)(i|cc)\b', title_lower)
        if cc_match:
            variant_parts.append(f"{cc_match.group(1)}{cc_match.group(2)}")
        
        return " ".join(variant_parts) if variant_parts else ""
    
    # Áp dụng trích xuất variant từ title
    df['variant_updated'] = df.apply(
        lambda row: extract_variant_from_title(row['title']) if row['model_updated'] != "Dòng khác" else "", 
        axis=1
    )
    
    # Chuẩn hóa brand_name
    def normalize_brand(brand):
        if pd.isna(brand):
            return brand
        
        brand = brand.strip().upper()
        
        # Map các tên brand phổ biến
        brand_mapping = {
            'HONDA': 'Honda',
            'YAMAHA': 'Yamaha',
            'SUZUKI': 'Suzuki',
            'SYM': 'SYM',
            'KAWASAKI': 'Kawasaki',
            'DUCATI': 'DUCATI',
            'BMW': 'BMW',
            'HARLEY-DAVIDSON': 'HARLEY-DAVIDSON',
            'TRIUMPH': 'TRIUMPH',
            'KTM': 'KTM',
            'PIAGGIO': 'PIAGGIO',
            'VESPA': 'PIAGGIO',
            'BENELLI': 'Benelli',
            'ROYAL ENFIELD': 'Royal Enfield'
        }
        
        for key, value in brand_mapping.items():
            if key in brand:
                return value
        
        return brand.title()  # Trả về brand với ký tự đầu viết hoa
    
    df['brand_normalized'] = df['brand'].apply(normalize_brand)
    
    return df

# Main execution
if __name__ == "__main__":
    # Đường dẫn đến file
    used_bike_path = "../input_xe_cu.csv"
    output_path = "processed_used_bikes.csv"
    
    # Xử lý dữ liệu xe cũ
    used_bike_df = process_used_bike_data(used_bike_path)
    
    # Lưu kết quả ra file CSV
    used_bike_df.to_csv(output_path, index=False)
    print(f"Đã lưu dữ liệu xe cũ đã xử lý vào file {output_path}")

Đọc file ../input_xe_cu.csv thành công: 16135 dòng
Đã lưu dữ liệu xe cũ đã xử lý vào file processed_used_bikes.csv


In [3]:
import pandas as pd
import numpy as np
import json
from datetime import datetime

def enrich_used_bike_data(used_bike_df, info_df):
    """
    Làm giàu dữ liệu xe cũ với thông tin từ catalog và price data
    """
    # Sao chép DataFrame để tránh thay đổi dữ liệu gốc
    enriched_df = used_bike_df.copy()
    
    # Tạo dictionary để lookup thông tin nhanh
    info_dict = {}
    for _, row in info_df.iterrows():
        brand = row['brand']
        model = row['model']
        variant = row['variant']
        key = (brand, model, variant)
        info_dict[key] = row.to_dict()
    
    # Đọc model_avg_prices
    avg_prices_df = pd.read_csv('catalog_price_info_avg_prices.csv')
    model_avg_prices = {}
    for _, row in avg_prices_df.iterrows():
        brand = row['brand']
        model = row['model']
        avg_price = row['average_price']
        model_key = (brand, model)
        model_avg_prices[model_key] = {'avg_price': avg_price}
    
    # Thêm các cột mới
    enriched_df['catalog_category'] = None
    enriched_df['catalog_years'] = None
    enriched_df['new_price_actual'] = None  # Giá xe mới thực tế từ nguồn
    enriched_df['new_price_model_avg'] = None  # Giá trung bình của model
    enriched_df['new_price'] = None  # Giá cuối cùng (ưu tiên giá thực tế)
    enriched_df['new_price_source'] = None  # Nguồn của giá (actual hoặc model_avg)
    
    # Cập nhật thông tin từ info_dict
    for idx, row in enriched_df.iterrows():
        brand = row['brand_normalized']
        model = row['model_updated']
        variant = row['variant_updated']
        
        # Tìm khớp chính xác (brand, model, variant)
        key = (brand, model, variant)
        if key in info_dict:
            info = info_dict[key]
            enriched_df.loc[idx, 'catalog_category'] = info['category']
            enriched_df.loc[idx, 'catalog_years'] = info['years']
            enriched_df.loc[idx, 'new_price_actual'] = info['new_price']
            enriched_df.loc[idx, 'new_price_model_avg'] = info['new_price_model_avg']
            
            # Xác định giá cuối cùng và nguồn
            if info['new_price'] is not None:
                enriched_df.loc[idx, 'new_price'] = info['new_price']
                enriched_df.loc[idx, 'new_price_source'] = 'actual'
            elif info['new_price_model_avg'] is not None:
                enriched_df.loc[idx, 'new_price'] = info['new_price_model_avg']
                enriched_df.loc[idx, 'new_price_source'] = 'model_avg'
        else:
            # Không tìm thấy khớp chính xác, thử tìm bằng (brand, model)
            model_key = (brand, model)
            if model_key in model_avg_prices:
                enriched_df.loc[idx, 'new_price_model_avg'] = model_avg_prices[model_key]['avg_price']
                enriched_df.loc[idx, 'new_price'] = model_avg_prices[model_key]['avg_price']
                enriched_df.loc[idx, 'new_price_source'] = 'model_avg'
    
    # Tính tỷ lệ giá xe cũ so với xe mới
    enriched_df['price_ratio'] = None
    mask = (enriched_df['price_value'].notna()) & (enriched_df['new_price'].notna())
    enriched_df.loc[mask, 'price_ratio'] = enriched_df.loc[mask, 'price_value'] / enriched_df.loc[mask, 'new_price']
    
    # Thêm nhận xét về giá
    def price_comment(row):
        if pd.isna(row['price_ratio']):
            return None
        
        ratio = row['price_ratio']
        if ratio < 0.3:
            return "Rất rẻ so với giá mới"
        elif ratio < 0.5:
            return "Rẻ so với giá mới"
        elif ratio < 0.7:
            return "Giá trung bình"
        elif ratio < 0.9:
            return "Khá cao so với giá mới"
        else:
            return "Rất cao so với giá mới"
    
    enriched_df['price_comment'] = enriched_df.apply(price_comment, axis=1)
    
    # Ước tính năm sử dụng dựa trên reg_year
    current_year = datetime.now().year
    enriched_df['years_used'] = None
    
    # Đảm bảo reg_year là kiểu số
    try:
        enriched_df['reg_year'] = pd.to_numeric(enriched_df['reg_year'], errors='coerce')
        mask = enriched_df['reg_year'].notna()
        enriched_df.loc[mask, 'years_used'] = current_year - enriched_df.loc[mask, 'reg_year']
    except Exception as e:
        print(f"Lỗi khi tính years_used: {e}")
    
    # Tính khấu hao trung bình mỗi năm
    enriched_df['yearly_depreciation'] = None
    mask = (enriched_df['years_used'].notna()) & (enriched_df['years_used'] > 0) & (enriched_df['price_ratio'].notna())
    enriched_df.loc[mask, 'yearly_depreciation'] = (1 - enriched_df.loc[mask, 'price_ratio']) / enriched_df.loc[mask, 'years_used']
    
    return enriched_df

# Main execution
if __name__ == "__main__":
    # Đọc file kết quả từ part1 và part2
    info_df = pd.read_csv('catalog_price_info.csv')
    used_bike_df = pd.read_csv('processed_used_bikes.csv')
    
    # Làm giàu dữ liệu xe cũ
    enriched_df = enrich_used_bike_data(used_bike_df, info_df)
    
    # Lưu kết quả cuối cùng
    enriched_df.to_csv('enriched_xe_cu_final.csv', index=False)
    print(f"Đã lưu dữ liệu xe cũ đã làm giàu vào file 'enriched_xe_cu_final.csv'")

Đã lưu dữ liệu xe cũ đã làm giàu vào file 'enriched_xe_cu_final.csv'
